# nDSM Generator **Batch** (DSM âˆ’ DGM1 / Bavaria)

This notebook downloads a **DTM (DGM1)** from Bavaria's INSPIRE **WCS 2.0.1**, clips it to your DSM's AOI, **aligns** it to the DSM grid, and computes an **nDSM = DSM âˆ’ DTM** â€” **for multiple DSMs in a folder**.

---

#### What you'll need
- A **folder containing DSM GeoTIFFs** (any CRS; will be handled).
- **Output folder** where products (DTMs and nDSMs) will be saved.
- **Credentials** for the Bavarian WCS (LDBV).
- Packages: `requests`, `rasterio`, `shapely`, `numpy`.

> **Data source & license:** DGM1 â€” *Bayerische Vermessungsverwaltung* (LDBV), **CC BY 4.0**. Bitte die Quelle in Ergebnissen angeben.

---

#### Inputs you set in the next cell
- `DSM_FOLDER` â€“ path to folder containing your DSM `.tif` files
- `OUTPUT_FOLDER` â€“ path where all products (DTMs and nDSMs) will be saved
- `DSM_PATTERN` â€“ file pattern to match (e.g., `"*.tif"` or `"*_DOM.tif"`)
- `BUFFER_M` â€“ small edge buffer (e.g., 0â€“2 m) to avoid cut-edge artifacts
- `PIXEL_M` â€“ requested output spacing from WCS (DGM1 native is 1 m)
- `GENERATE_NDSM` â€“ set to `True` to generate nDSM files, `False` to only download/align DTMs

---

#### Outputs produced (per DSM)
- `{basename}_DTM.tif` â€” raw DTM from WCS
- `{basename}_DTM_aligned_to_DSM.tif` â€” DTM snapped to **exact DSM grid** (same CRS, transform, size)
- `{basename}_nDSM.tif` â€” height of objects above ground (if `GENERATE_NDSM = True`)

---

#### How it works (pipeline)
1. Scan `DSM_FOLDER` for DSM files matching pattern.
2. For each DSM:
   - Read DSM extent â†’ transform to **EPSG:25832** â†’ apply small buffer.
   - WCS 2.0.1 **GetCoverage** for `EL.ElevationGridCoverage` with that AOI.
   - **Align** DTM to the DSM's pixel grid (CRS, transform, dimensions).
   - Compute **nDSM = DSM âˆ’ aligned DTM** (if enabled).
3. Save all products to `OUTPUT_FOLDER`.

---

#### Credentials (recommended: env vars)
Set once in your conda env so you don't type them each time.

**Anaconda:**
```bash
conda activate nDSM_generator
conda env config vars set LDBV_USER="your_user" LDBV_PASS="your_password"
conda deactivate && conda activate nDSM_generator
```

In [14]:
from modules.ndsm_tools import *
import os
import getpass
from pathlib import Path
from glob import glob

In [15]:
# === USER INPUTS ===
DSM_FOLDER = r"D:\DSMs"  # Folder containing DSM files
OUTPUT_FOLDER = r"D:\CHMs"  # Where to save all products
DTM_OUTPUT_FOLDER = r"D:\DTMs"  # Where to save DTM products (raw + aligned)
DSM_PATTERN = "*_DSM.tif"  # Pattern to match DSM files (e.g., "*.tif" or "*_DSM.tif")

BUFFER_M = 2.0   # 0..2 m is fine for most use cases
PIXEL_M  = 1.0   # DGM1 native spacing (m)
GENERATE_NDSM = True  # Set to False if you only want DTMs
RESAMPLING = "bilinear"  # Options: "bilinear" (recommended), "cubic" (highest quality), "nearest" (preserves values)\n

# === Credentials ===
USER = os.getenv("LDBV_USER") or input("LDBV username: ")
PWD  = os.getenv("LDBV_PASS") or getpass.getpass("LDBV password: ")

In [16]:
# === BATCH PROCESSING ===

# Create output folders if they don't exist
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
os.makedirs(DTM_OUTPUT_FOLDER, exist_ok=True)
print(f"CHM output folder: {OUTPUT_FOLDER}")
print(f"DTM output folder: {DTM_OUTPUT_FOLDER}\n")

# Find all DSM files in the input folder
dsm_search_path = os.path.join(DSM_FOLDER, DSM_PATTERN)
dsm_files = glob(dsm_search_path)

if not dsm_files:
    print(f"❌ No DSM files found matching pattern '{DSM_PATTERN}' in {DSM_FOLDER}")
else:
    print(f"✓ Found {len(dsm_files)} DSM file(s) to process:\n")
    for i, dsm_file in enumerate(dsm_files, 1):
        print(f"  {i}. {os.path.basename(dsm_file)}")
    
    print(f"\n{'='*80}")
    print("Starting batch processing...")
    print(f"{'='*80}\n")
    
    # Process each DSM
    results = []
    errors = []
    
    for i, dsm_path in enumerate(dsm_files, 1):
        dsm_basename = os.path.splitext(os.path.basename(dsm_path))[0]
        print(f"\n[{i}/{len(dsm_files)}] Processing: {dsm_basename}")
        print("-" * 80)
        
        try:
            # Define output paths - replace '_DSM' with '_DTM' or '_CHM'
            dtm_basename = dsm_basename.replace('_DSM', '_DTM') if '_DSM' in dsm_basename else f"{dsm_basename}_DTM"
            chm_basename = dsm_basename.replace('_DSM', '_CHM') if '_DSM' in dsm_basename else f"{dsm_basename}_CHM"
            
            dtm_save_path = os.path.join(DTM_OUTPUT_FOLDER, f"{dtm_basename}.tif")
            ndsm_save_path = os.path.join(OUTPUT_FOLDER, f"{chm_basename}.tif") if GENERATE_NDSM else None
            
            # Run the pipeline
            aligned_dtm, ndsm = run_pipeline(
                dsm_path=dsm_path,
                dtm_save_path=dtm_save_path,
                ndsm_save_path=ndsm_save_path,
                buffer_m=BUFFER_M,
                pixel_m=PIXEL_M,
                resampling=RESAMPLING,
                user=USER,
                password=PWD,
                allow_prompt=False,  # we already supplied creds above
                dtm_output_folder=DTM_OUTPUT_FOLDER
            )
            
            results.append({
                'dsm': dsm_basename,
                'aligned_dtm': aligned_dtm,
                'ndsm': ndsm,
                'status': '✓ Success'
            })
            
            print(f"\n✓ Completed {dsm_basename}")
            print(f"  - Aligned DTM: {aligned_dtm}")
            if ndsm:
                print(f"  - CHM: {ndsm}")
                
        except Exception as e:
            error_msg = str(e)
            errors.append({
                'dsm': dsm_basename,
                'error': error_msg
            })
            print(f"\n❌ Error processing {dsm_basename}: {error_msg}")
    
    # Summary
    print(f"\n\n{'='*80}")
    print("BATCH PROCESSING SUMMARY")
    print(f"{'='*80}")
    print(f"\nTotal files: {len(dsm_files)}")
    print(f"Successful: {len(results)}")
    print(f"Failed: {len(errors)}")
    
    if results:
        print(f"\n✓ Successfully processed:")
        for r in results:
            print(f"  - {r['dsm']}")
    
    if errors:
        print(f"\n❌ Failed to process:")
        for e in errors:
            print(f"  - {e['dsm']}: {e['error']}")
    
    print(f"\nCHM products saved to: {OUTPUT_FOLDER}")
    print(f"DTM products saved to: {DTM_OUTPUT_FOLDER}")
    print(f"\n{'='*80}\n")


CHM output folder: D:\CHMs
DTM output folder: D:\DTMs

❌ No DSM files found matching pattern '*_DSM.tif' in D:\DSMs
